# DUAW recomender system

In [17]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [18]:
def display_all(df):
    with pd.option_context('display.max_rows',None,'display.max_columns', None):
        display(df)  

## API request Repository names and description
https://www.nylas.com/blog/use-python-requests-module-rest-apis/

In [53]:
import requests

In [54]:
url = "https://api.github.com/orgs/faketechlabsberlin/repos"
repos = requests.get(url)
repos_json = repos.json()
print(type(repos_json))

<class 'list'>


In [55]:
#Authentification for the get API
#requests.get('https://api.github.com/user', auth=HTTPBasicAuth('username', 'password'))

In [56]:
data_df = pd.DataFrame(repos_json)

### Similarity analysis

In [57]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_similarity

In [58]:
corpus = data_df.description

In [59]:
repo_names = data_df.name

#### vectorization

In [60]:
vectorizer = CountVectorizer() # numrac list
features = vectorizer.fit_transform(corpus).todense() # each of these documents, in list of numbers

##### similarity analysis for all repositories

In [61]:
b= np.zeros( (len(features), len(features)) )

In [62]:
x=-1
y=-1
for each in features:
    x=x+1
    for f in features:
        y=y+1
        value = cosine_similarity(each, f) # distance between documents
        b[y][x]=value
    y=-1
#print(a)
d_df = pd.DataFrame(b)

/Users/constanze/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
/Users/constanze/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
/Users/constanze/opt/anaconda3/l

##### return the names of most similar to least similar repositories for each repository

In [63]:
list_of_lists =[]
for each in d_df.columns: # for each column in data frame
    list_series=[]
    current_series = d_df[each]
    sorted_series = current_series.sort_values(ascending=False)
   # return list with repositoryies
    for each in sorted_series.index:
        list_series.append(repo_names[each]) # append list 
    list_of_lists.append(list_series)

In [64]:
df = pd.DataFrame(list_of_lists)
highest_results=df.iloc[:,0:4]
highest_results

,0,1,2,3
0,test-repository-1,st21-jobzilla,st21-what-the-wine,Travel-20
1,st21-terraViz,st21-jobzilla,st21-what-street-art,st21-what-the-wine
2,st21-stress-tracker,st21-artifacts,st21-what-street-art,Travel-20
3,st21-what-the-wine,st21-jobzilla,st21-what-street-art,Travel-20
4,st21-jobzilla,st21-what-the-wine,st21-terraViz,st21-beachblue
5,st21-artifacts,st21-what-street-art,migr-AI-tion,Travel-20
6,st21-busking-radar,Travel-20,st21-what-street-art,st21-artifacts
7,st21-what-street-art,st21-artifacts,Travel-20,st21-stress-tracker
8,st21-beachblue,st21-jobzilla,st21-busking-radar,st21-what-the-wine
9,Travel-20,st21-artifacts,migr-AI-tion,st21-what-street-art


## export results in JSON files

In [65]:
# Convert DataFrame to JSON
data = highest_results.to_json('./recomendations.json', orient='values')
print(data)

None
